In [1]:
# Fix
from run import main

In [2]:
temporal_sets = main(log=True)

# Deleted the existing log file.

# Loading the data...
    >>> SUCCESS!

# Converting the data into preconfigured dtypes...
    >>> SUCCESS!

#########################################################

Summary for the loaded dataset

The total number of rows is 124976

The total number of rows is 25

Descriptive Statistics:

       school_ncesid  school_latitude  school_longitude  \
count   1.157430e+05    124976.000000     124976.000000   
mean    2.448448e+11        36.827284        -95.859299   
std     1.644728e+11         4.963669         18.392876   
min     1.000050e+10        18.249140       -171.690554   
25%     6.344101e+10        33.872504       -117.806418   
50%     2.200870e+11        36.617410        -90.101563   
75%     3.704880e+11        40.676156        -80.713740   
max     6.100010e+11        65.672562        -66.628036   

       total_price_including_optional_support  students_reached  
count                           124976.000000     124917.000000  
mean     

In [10]:
train_df = temporal_sets[0][1][0]
test_df = temporal_sets[0][1][1]

In [7]:
temporal_sets[0][1][2]

projectid
0000fa3aa8f6649abab23615b546016d    1
000134f07d4b30140d63262c871748ff    0
000be0b57987d6c29d650a88eac2961b    0
000e6a13fd69bf0e426403b1d5da0e99    1
00105edf14ebb7f146d6ba2098639bb1    0
00134d14226ef613134712c54211f754    0
00136585c0e4a67afaa964403cf9f45b    0
0013a402fb4cf84e3d60d7a59566dff8    1
0014d10f538cd1a7d97053f4d653f1f6    1
00161a60318f303f71bce3e292aa547d    1
001706ada83ac3f90369cdd3915d2dcf    1
00176778e93d54ed26f601f7a0af4b35    1
00199bf192b852aa21573c69ed14565e    1
001d5055cf26a15651c3def86c6d6372    1
00239094b2720ccbd7a456a3c962f03b    0
0023f4e20e9ac0d53d7f1436d5711c5b    1
0024f9ac77374c0aac865543bb78eb46    1
00250a8ddc3f0223ac383aeeaa48c2f1    0
00253a9de723379d8b86d5b08b7d46ff    0
0027e1923d1ea12fccb705e50db911dc    1
002e341587b92b8963dd7b97cfdf7129    1
002f926c33d27d1a5e917186b786335d    1
003257df1a85fa0f0fefdb0fd6e55237    1
0035903dc05abc095cb4663a5fc9ac53    1
00377c2e78f1aa128fc1a509ed7b9213    0
003a0f03bd7f9114bf5b7357fc141277    1
00

In [11]:
print(test_df['date_posted'].min())
print(test_df['date_posted'].max())
print(train_df['date_posted'].min())
print(train_df['date_posted'].max())

2012-07-01 00:00:00
2012-12-31 00:00:00
2012-01-01 00:00:00
2012-06-30 00:00:00
